In [266]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('indonesian')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.corpus.stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading indonesian: Package 'indonesian' not found
[nltk_data]     in index
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [267]:
!dir

 Volume in drive D is Data
 Volume Serial Number is 86AA-243D

 Directory of d:\Binar Academy - Data Science\challenge_platinum\challenge_platinum_binar_academy

20/02/2023  21:51    <DIR>          .
18/02/2023  23:45    <DIR>          ..
21/02/2023  15:24            79.389 challenge_platinum.ipynb
15/04/2020  14:27         1.858.473 data.csv
20/02/2023  21:59             1.611 LSTM_catatan.txt
20/02/2023  20:54             2.881 peningkatan_akurasi_model_neuralNetwork.txt
22/02/2023  18:03        61.091.608 sentiment_analysis_model_challenge.h5
16/02/2023  19:18         2.186.718 train_preprocess.tsv.txt
               6 File(s)     65.220.680 bytes
               2 Dir(s)  157.665.898.496 bytes free


In [268]:
df = pd.read_csv("train_preprocess.tsv.txt", sep="\t", engine="python", names=["data", "label"])

In [269]:
df

,data,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [270]:
df.shape

(11000, 2)

In [271]:
def cleansing(text):
    # Make sentence being lowercase
    text = text.lower()

    # Remove user, rt, \n, retweet, \t, url, xd
    pattern_1 = r'(user|retweet|\\t|\\r|url|xd)'
    text = re.sub(pattern_1, '', text)

    # Remove mention
    pattern_2 = r'@[^\s]+'
    text = re.sub(pattern_2, '', text)

    # Remove hashtag
    pattern_3 = r'#([^\s]+)'
    text = re.sub(pattern_3, '', text)

    # Remove general punctuation, math operation char, etc.
    pattern_4 = r'[\,\@\*\_\-\!\:\;\?\'\.\"\)\(\{\}\<\>\+\%\$\^\#\/\`\~\|\&\|]'
    text = re.sub(pattern_4, ' ', text)

    # Remove single character
    pattern_5 = r'\b\w{1,3}\b'
    text = re.sub(pattern_5, '', text)

    # Remove emoji
    pattern_6 = r'\\[a-z0-9]{1,5}'
    text = re.sub(pattern_6, '', text)

    # Remove digit character
    pattern_7 = r'\d+'
    text = re.sub(pattern_7, '', text)

    # Remove url start with http or https
    pattern_8 = r'(https|https:)'
    text = re.sub(pattern_8, '', text)

    # Remove (\); ([); (])
    pattern_9 = r'[\\\]\[]'
    text = re.sub(pattern_9, '', text)

    # Remove character non ASCII
    pattern_10 = r'[^\x00-\x7f]'
    text = re.sub(pattern_10, '', text)

    # Remove character non ASCII
    pattern_11 = r'(\\u[0-9A-Fa-f]+)'
    text = re.sub(pattern_11, '', text)

    # Remove multiple whitespace
    pattern_12 = r'(\s+|\\n)'
    text = re.sub(pattern_12, ' ', text)
    
    # Remove whitespace at the first and end sentences
    text = text.rstrip()
    text = text.lstrip()
    return text

# indo_stop_words = stopwords.words("indonesian")

#def remove_stopwords(text):
    #return ' '.join([word for word in word_tokenize(text) if word not in indo_stop_words])

def tokenisasi(text):
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens

def stemming(tokens):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return ' '.join([stemmer.stem(token) for token in tokens])

In [272]:
df_coba = df.applymap(cleansing)
# df_coba = df_coba.applymap(remove_stopwords)
df_coba

,data,label
0,warung dimiliki oleh pengusaha pabrik tahu yan...,positive
1,mohon ulama lurus k mmbri hujjah partai yang h...,neutral
2,lokasi strategis jalan sumatera bandung tempat...,positive
3,betapa bahagia diri saat unboxing paket barang...,positive
4,jadi mahasiswa jangan sombong dong kasih kartu...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan apalagi kepiting yang menyen...,positive
10997,hormati partai partai yang telah berkoalisi,neutral
10998,pagi pagi pasteur sudah macet parah bikin jeng...,negative


In [273]:
df_coba['word_tokens'] = df_coba['data'].apply(tokenisasi)
df_coba

,data,label,word_tokens
0,warung dimiliki oleh pengusaha pabrik tahu yan...,positive,"[warung, dimiliki, oleh, pengusaha, pabrik, ta..."
1,mohon ulama lurus k mmbri hujjah partai yang h...,neutral,"[mohon, ulama, lurus, k, mmbri, hujjah, partai..."
2,lokasi strategis jalan sumatera bandung tempat...,positive,"[lokasi, strategis, jalan, sumatera, bandung, ..."
3,betapa bahagia diri saat unboxing paket barang...,positive,"[betapa, bahagia, diri, saat, unboxing, paket,..."
4,jadi mahasiswa jangan sombong dong kasih kartu...,negative,"[jadi, mahasiswa, jangan, sombong, dong, kasih..."
...,...,...,...
10995,tidak kecewa,positive,"[tidak, kecewa]"
10996,enak rasa masakan apalagi kepiting yang menyen...,positive,"[enak, rasa, masakan, apalagi, kepiting, yang,..."
10997,hormati partai partai yang telah berkoalisi,neutral,"[hormati, partai, partai, yang, telah, berkoal..."
10998,pagi pagi pasteur sudah macet parah bikin jeng...,negative,"[pagi, pagi, pasteur, sudah, macet, parah, bik..."


In [274]:
df_coba[['data']][0:10]

,data
0,warung dimiliki oleh pengusaha pabrik tahu yan...
1,mohon ulama lurus k mmbri hujjah partai yang h...
2,lokasi strategis jalan sumatera bandung tempat...
3,betapa bahagia diri saat unboxing paket barang...
4,jadi mahasiswa jangan sombong dong kasih kartu...
5,makanan beragam harga makanan food stall akan ...
6,pakai kartu kredit tidak untung malah rugi besar
7,tempat unik bagus buat foto makanan enak pegaw...
8,saya bersama keluarga baru saja menikmati peng...
9,bersyukur


In [275]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm

In [276]:
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()

# df_coba['stemmed_word_tokens'] = ''

# # apply stemming to each sentence in the list
# for i in tqdm(range(len(df_coba['word_tokens']))):
#     stemmed_words = [stemmer.stem(word) for word in df_coba['word_tokens'][i]]
#     df_coba.at[i, 'stemmed_word_tokens'] = ' '.join(stemmed_words)

# # print the original and stemmed sentences
# print(df_coba[['word_tokens', 'stemmed_word_tokens']])

## One Hot Encoding

In [277]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_coba['label_encode'] = le.fit_transform(df_coba['label'])

display(df_coba.head())

print(df_coba.dtypes)

,data,label,word_tokens,label_encode
0,warung dimiliki oleh pengusaha pabrik tahu yan...,positive,"[warung, dimiliki, oleh, pengusaha, pabrik, ta...",2
1,mohon ulama lurus k mmbri hujjah partai yang h...,neutral,"[mohon, ulama, lurus, k, mmbri, hujjah, partai...",1
2,lokasi strategis jalan sumatera bandung tempat...,positive,"[lokasi, strategis, jalan, sumatera, bandung, ...",2
3,betapa bahagia diri saat unboxing paket barang...,positive,"[betapa, bahagia, diri, saat, unboxing, paket,...",2
4,jadi mahasiswa jangan sombong dong kasih kartu...,negative,"[jadi, mahasiswa, jangan, sombong, dong, kasih...",0


data            object
label           object
word_tokens     object
label_encode     int32
dtype: object


In [278]:
df_coba.isna().sum()

data            0
label           0
word_tokens     0
label_encode    0
dtype: int64

In [279]:
# Menggabungkan setiap elemen dalam kolom 'word_tokens' menjadi satu string
df_coba['string_tokens'] = df_coba['word_tokens'].apply(' '.join)

In [280]:
df_coba

,data,label,word_tokens,label_encode,string_tokens
0,warung dimiliki oleh pengusaha pabrik tahu yan...,positive,"[warung, dimiliki, oleh, pengusaha, pabrik, ta...",2,warung dimiliki oleh pengusaha pabrik tahu yan...
1,mohon ulama lurus k mmbri hujjah partai yang h...,neutral,"[mohon, ulama, lurus, k, mmbri, hujjah, partai...",1,mohon ulama lurus k mmbri hujjah partai yang h...
2,lokasi strategis jalan sumatera bandung tempat...,positive,"[lokasi, strategis, jalan, sumatera, bandung, ...",2,lokasi strategis jalan sumatera bandung tempat...
3,betapa bahagia diri saat unboxing paket barang...,positive,"[betapa, bahagia, diri, saat, unboxing, paket,...",2,betapa bahagia diri saat unboxing paket barang...
4,jadi mahasiswa jangan sombong dong kasih kartu...,negative,"[jadi, mahasiswa, jangan, sombong, dong, kasih...",0,jadi mahasiswa jangan sombong dong kasih kartu...
...,...,...,...,...,...
10995,tidak kecewa,positive,"[tidak, kecewa]",2,tidak kecewa
10996,enak rasa masakan apalagi kepiting yang menyen...,positive,"[enak, rasa, masakan, apalagi, kepiting, yang,...",2,enak rasa masakan apalagi kepiting yang menyen...
10997,hormati partai partai yang telah berkoalisi,neutral,"[hormati, partai, partai, yang, telah, berkoal...",1,hormati partai partai yang telah berkoalisi
10998,pagi pagi pasteur sudah macet parah bikin jeng...,negative,"[pagi, pagi, pasteur, sudah, macet, parah, bik...",0,pagi pagi pasteur sudah macet parah bikin jeng...


In [281]:
df_copy = df_coba.copy()
df_copy.drop(columns=["data", "label"], inplace=True)

## NAIVE BAYES

In [282]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

vectorizer = CountVectorizer()

X_train, X_test, y_train, y_test = train_test_split(df_copy['word_tokens'], df_copy['label_encode'], train_size=0.8, random_state=42)
X_train = [" ".join(tokens) for tokens in X_train]
X_test = [" ".join(tokens) for tokens in X_test]
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [283]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

In [284]:
from sklearn.metrics import confusion_matrix, confusion_matrix, classification_report
confusion_matrix(y_pred,y_test)

array([[ 528,   69,  116],
       [   5,  135,    7],
       [ 147,   35, 1158]], dtype=int64)

In [285]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.78      0.74      0.76       713
           1       0.56      0.92      0.70       147
           2       0.90      0.86      0.88      1340

    accuracy                           0.83      2200
   macro avg       0.75      0.84      0.78      2200
weighted avg       0.84      0.83      0.83      2200



## LSTM

In [286]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

sentences = df_coba['word_tokens'].to_list()
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(sentences)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of Data Tensor : ', X.shape)

Found 15763 unique tokens.
Shape of Data Tensor :  (11000, 250)


In [287]:
Y = pd.get_dummies(df_copy['label_encode']).values
print('Shape of Label Tensor : ', Y.shape)

Shape of Label Tensor :  (11000, 3)


In [288]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.10, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(9900, 250) (9900, 3)
(1100, 250) (1100, 3)


In [289]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 2
batch_size = 64

model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/2
140/140 [==============================] - 147s 1s/step - loss: 0.6305 - accuracy: 0.7396 - val_loss: 0.4550 - val_accuracy: 0.8232
Epoch 2/2
140/140 [==============================] - 145s 1s/step - loss: 0.3178 - accuracy: 0.8863 - val_loss: 0.3550 - val_accuracy: 0.8768


In [290]:
y_pred = model.predict(X_test)
y_pred.shape

35/35 [==============================] - 4s 90ms/step


(1100, 3)

In [291]:
import numpy as np

y_pred_labels = np.argmax(y_pred, axis=1)
print(y_pred_labels)

[0 1 2 ... 2 2 0]


In [292]:
from sklearn.metrics import accuracy_score

y_pred_classes = np.argmax(y_pred, axis=1)
Y_test_classes = np.argmax(Y_test, axis=1)
accuracy = accuracy_score(Y_test_classes, y_pred_classes)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 87.36%


In [293]:
from sklearn.metrics import classification_report

y_pred_classes = np.argmax(y_pred, axis=1)
Y_test_classes = np.argmax(Y_test, axis=1)
print(classification_report(Y_test_classes, y_pred_classes))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83       342
           1       0.85      0.75      0.80       116
           2       0.91      0.91      0.91       642

    accuracy                           0.87      1100
   macro avg       0.86      0.84      0.85      1100
weighted avg       0.87      0.87      0.87      1100



## Save Model

In [294]:
model.save("sentiment_analysis_model_challenge.h5")

## Cara menggunakan model kembali dengan load model

In [295]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import load_model

# load data baru
df_new = pd.read_csv("train_preprocess.tsv.txt", sep="\t", engine="python", names=["data", "label"])
df_new = df_new.applymap(cleansing)
# df_new = df_new.applymap(remove_stopwords)

# drop kolom label
df_new.drop(columns=['label'], inplace=True)

sentences = df_new['data'].to_list()

# lakukan preprocessing pada data baru
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(sentences)
X_new = tokenizer.texts_to_sequences(sentences)
X_new = pad_sequences(X_new, maxlen=MAX_SEQUENCE_LENGTH)

loaded_model = load_model("D:\Binar Academy - Data Science\challenge_platinum\challenge_platinum_binar_academy\sentiment_analysis_model_challenge.h5")

# lakukan prediksi pada data baru
y_prob = loaded_model.predict(X_new)
y_pred = y_prob.argmax(axis=-1)

# konversi nilai prediksi menjadi label sentimen
labels = {0: "negative", 1: "neutral", 2: "positive"}
df_new["label_sentimen"] = [labels[pred] for pred in y_pred]

344/344 [==============================] - 32s 92ms/step


In [296]:
df_new

,data,label_sentimen
0,warung dimiliki oleh pengusaha pabrik tahu yan...,positive
1,mohon ulama lurus k mmbri hujjah partai yang h...,negative
2,lokasi strategis jalan sumatera bandung tempat...,positive
3,betapa bahagia diri saat unboxing paket barang...,positive
4,jadi mahasiswa jangan sombong dong kasih kartu...,negative
...,...,...
10995,tidak kecewa,negative
10996,enak rasa masakan apalagi kepiting yang menyen...,positive
10997,hormati partai partai yang telah berkoalisi,neutral
10998,pagi pagi pasteur sudah macet parah bikin jeng...,negative


## Cara menggunakan kembali model yang telah dibuat (load model)

In [297]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import load_model

# load data baru
df_new = pd.read_csv("data.csv", encoding='latin-1')
df_new['Tweet'] = df_new['Tweet'].apply(cleansing)
# df_new['Tweet'] = df_new['Tweet'].apply(remove_stopwords)

sentences = df_new['Tweet'].to_list()

# lakukan preprocessing pada data baru
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(sentences)
X_new = tokenizer.texts_to_sequences(sentences)
X_new = pad_sequences(X_new, maxlen=MAX_SEQUENCE_LENGTH)

loaded_model = load_model("D:\Binar Academy - Data Science\challenge_platinum\challenge_platinum_binar_academy\sentiment_analysis_model_challenge.h5")

# lakukan prediksi pada data baru
y_prob = loaded_model.predict(X_new)
y_pred = y_prob.argmax(axis=-1)

# konversi nilai prediksi menjadi label sentimen
labels = {0: "negative", 1: "neutral", 2: "positive"}
df_new["label_sentimen"] = [labels[pred] for pred in y_pred]

412/412 [==============================] - 35s 85ms/step


In [298]:
df_tweet_predict = pd.DataFrame({"Tweet": df_new['Tweet'],
                           "Label": df_new['label_sentimen']
                        })
df_tweet_predict.head()

,Tweet,Label
0,disaat semua cowok berusaha melacak perhatian ...,positive
1,siapa yang telat ngasih edan sarap bergaul den...,positive
2,kadang berfikir kenapa tetap percaya pada tuha...,positive
3,matamu sipit tapi diliat dari mana,positive
4,kaum cebong kapir udah keliatan dongoknya dari...,negative


In [299]:
df_tweet_predict

,Tweet,Label
0,disaat semua cowok berusaha melacak perhatian ...,positive
1,siapa yang telat ngasih edan sarap bergaul den...,positive
2,kadang berfikir kenapa tetap percaya pada tuha...,positive
3,matamu sipit tapi diliat dari mana,positive
4,kaum cebong kapir udah keliatan dongoknya dari...,negative
...,...,...
13164,jangan asal ngomong ndasmu congor sekate anjyng,negative
13165,kasur mana enak kunyuk,negative
13166,hati hati bisu bosan huft,negative
13167,yang real mudah terdeteksi yang terkubur suatu...,positive


In [300]:
df_tweet_predict['Label'].unique()

array(['positive', 'negative', 'neutral'], dtype=object)